# <center> MATH&ML-6. Математический анализ в контексте задачи оптимизации. Часть III
---

### 2. Градиентный спуск: применение и модификации

In [157]:
# Задание 2.7
# Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии
# Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# Загрузите стандартный датасет об алмазах из библиотеки Seaborn:
import seaborn as sns
df = sns.load_dataset('diamonds')
# Удалите часть признаков:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
# Закодируйте категориальные признаки:
df = pd.get_dummies(df, drop_first=True)
# Логарифмируйте признаки:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])
# Определите целевую переменную и предикторы:
X = df.drop(columns="price")
y = df["price"]
# Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor)
reg = SGDRegressor()
# Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:
grid = {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 10),
    "l1_ratio": np.linspace(0, 1, 10),
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
    }

grid_search = GridSearchCV(
    estimator=reg,
    param_grid=grid,
    n_jobs=-1
    )
grid_search.fit(X_train, y_train)
y_test_pred = grid_search.predict(X_test)
round(metrics.mean_squared_error(y_test, y_test_pred), 3)

0.044

---
### 3. Метод Ньютона

In [158]:
x = 0.6296335078534031
f = 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
dx = 30*x**4 - 20*x**3 - 12*x**2 + 6*x
x - f / dx

0.6286680781673306

In [159]:
# Задание 3.1
def newton(x):
    def f(x):
        return 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
    def dx(x):
        return 30*x**4 - 20*x**3 - 12*x**2 + 6*x
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.001:
            print(round(x_new, 3))
            break
        x = x_new

newton(0.7)

0.629


In [160]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

In [161]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

#21.695121951219512
#11.734125501243229
#7.1123493600499685
#5.365000391507974
#5.015260627016227
#5.000029000201801
#5.000000000105126
#5.0

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [162]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)
# 5

5.0

In [163]:
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)
# 5

5.0

In [164]:
# Задание 3.6
def f(x):
    return x**3 - 72*x - 220

def dx(x):
    return 3*x**2 - 72

x0 = 12

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.001:
            print(round(x_new, 3))
            break
        x = x_new

newton(f, dx, x0)

9.727


In [165]:
# Задание 3.7
def f(x):
    return x**2 + 9*x - 5

def dx(x):
    return 2*x + 9

x0 = 2.2

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.01:
            print(round(x_new, 2))
            break
        x = x_new

newton(f, dx, x0)

0.52


In [166]:
# Задание 3.9
def dx(x):
    return 24*x**2 - 4*x

def ddx(x):
    return 48*x -4

x0 = 42

def newton(f, dx, x):
    while True:
        x_new = x - f(x) / dx(x)
        if abs(x - x_new) < 0.0001:
            print(round(x_new, 3))
            break
        x = x_new

newton(dx, ddx, x0)

0.167


---
### 4. Квазиньютоновские методы

Подгрузим необходимые библиотеки:

In [167]:
import numpy as np
from scipy.optimize import minimize

Определим функцию, которую будем оптимизировать. Вместо отдельных $x$ и $y$ можно взять координаты единого вектора:

In [168]:
def func(x):
    return x[0]**2.0 + x[1]**2.0

Теперь определим градиент для функции:

In [169]:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

Зададим начальную точку:

In [170]:
x_0 = [1.0, 1.0]

Определим алгоритм:

In [171]:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

Выведем результаты:

In [172]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
 
#Статус оптимизации Optimization terminated successfully.
#Количество оценок: 3
#Решение: f([0. 0.]) = 0.00000

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Итак, мы получили, что минимум функции достигается в точке $(0,0)$. Значение функции в этой точке также равно нулю.

Можно повторить то же самое с вариацией  *L-BFGS-B*:

In [173]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Результат будет тем же самым.

In [174]:
# Задание 4.1
def func(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20

def grad_func(x):
    return np.array([2*x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

x0 = [-400, -400]

result = minimize(func, x0, method='L-BFGS-B', jac=grad_func)
result

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: -0.9999999999999218
        x: [-4.000e+00  1.000e+00]
      nit: 4
      jac: [ 2.900e-07  2.724e-07]
     nfev: 9
     njev: 9
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [175]:
# Задание 4.4
def f(x):
    return x**2 - 3*x + 45

def dx(x):
    return 2*x - 3

x0 = 10

result = minimize(f, x0, method='BFGS', jac=dx)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 4
      jac: [ 0.000e+00]
 hess_inv: [[ 5.000e-01]]
     nfev: 5
     njev: 5

In [176]:
# Задание 4.5
minimize(f, x0, method='L-BFGS-B', jac=dx)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 42.75
        x: [ 1.500e+00]
      nit: 2
      jac: [ 0.000e+00]
     nfev: 3
     njev: 3
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [177]:
# Задание 4.7
def f(dot):
    x = dot[0]
    y = dot[1]
    return x**4 + 6*y**2 + 10

def grad_f(dot):
    x = dot[0]
    y = dot[1]
    return np.array([4*x**3, 12*y])

x0 = [100, 100]

result_1 = minimize(f, x0, method='BFGS', jac=grad_f)
result_2 = minimize(f, x0, method='L-BFGS-B', jac=grad_f)
print(f'BFGS: \n {result_1} \n\n\nL-BFGS-B:  \n {result_2}')

BFGS: 
   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 10.000000030008898
        x: [ 1.316e-02  6.653e-14]
      nit: 34
      jac: [ 9.120e-06  7.984e-13]
 hess_inv: [[ 2.016e+02 -4.163e-09]
            [-4.163e-09  7.317e-02]]
     nfev: 37
     njev: 37 


L-BFGS-B:  
   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 10.00000000827103
        x: [-9.527e-03 -2.322e-06]
      nit: 37
      jac: [-3.459e-06 -2.786e-05]
     nfev: 40
     njev: 40
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


---
### 6. Практика: линейное программирование

**Пример № 1. SciPy (**`scipy.optimize.linprog`**)**

У нас есть 6 товаров с заданными ценами на них и заданной массой.

Вместимость сумки, в которую мы можем положить товары, заранее известна и равна 15 кг.

Какой товар и в каком объёме необходимо взять, чтобы сумма всех цен товаров была максимальной?

In [178]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [179]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [180]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

**Пример № 2. CVXPY**

Снова решим задачу из примера № 1, но уже предположим, что товары нельзя дробить, и будем решать задачу целочисленного линейного программирования.

*SciPy* не умеет решать такие задачи, поэтому будем использовать новую библиотеку *CVXPY*.

>**Важно!** С установкой этот библиотеки порой возникают проблемы. Если вы столкнулись с трудностями, посоветуйтесь с ментором или воспользуйтесь *Google Colaboratory*.

In [181]:
import cvxpy

In [182]:
x = cvxpy.Variable(shape=n, integer = True)

In [183]:
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

In [184]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [185]:
# Вообще со стадартным солвером должна быть бесконечность, но выдаёт ошибку
problem.solve(solver='ECOS_BB')

-138412039.00000018

In [186]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c @ x

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

print(-problem.solve())
print(x.value)

49.0
[-0. -0. -0.  7. -0.  0.]


А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём $x$ типа *boolean*.

$x=0$ или $x=1$

Программное решение такой задачи имеет следующий вид:

In [187]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c @ x

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

print(-problem.solve())
print(x.value)

17.0
[1. 0. 0. 1. 0. 1.]


> Обратите внимание, что, используя *SciPy*, мы могли не указывать явно, что $x$ только положительные, так как в линейном программировании считаются только неотрицательные $x$.

>А вот *CVXPY* универсальна. Мы просто задали функцию, не указывая, что это линейное программирование. *CVXPY* «поняла», что это задача оптимизации, и использовала нужные алгоритмы. Поэтому здесь ограничение на положительные $x$ мы указывали явно.

**Пример № 3. PuLP**

В нашей каршеринговой компании две модели автомобилей: модель $A$ и модель $B$. Автомобиль $A$ даёт прибыль в размере 20 тысяч в месяц, а автомобиль $B$ — 45 тысяч в месяц. Мы хотим заказать на заводе новые автомобили и максимизировать прибыль. Однако на производство и ввод в эксплуатацию автомобилей понадобится время:

* Проектировщику требуется 4 дня, чтобы подготовить документы для производства каждого автомобиля типа $A$, и 5 дней — для каждого автомобиля типа $B$.
* Заводу требуется 3 дня, чтобы изготовить модель $A$, и 6 дней, чтобы изготовить модель $B$.
* Менеджеру требуется 2 дня, чтобы ввести в эксплуатацию в компании автомобиль $A$, и 7 дней —  автомобиль $B$.
* Каждый специалист может работать суммарно 30 дней.

In [188]:
from pulp import *

In [189]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

c:\Users\Blooodniy\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


In [201]:
# Задание 6.1
c = np.array([2, 5, 3, 7, 7, 6])
A = np.array([
    [1, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1],
    [-1, 0, 0, -1, 0, 0], 
    [0, -1, 0, 0, -1, 0],
    [0, 0, -1, 0, 0, -1]
    ])
b = np.array([180, 220, -110, -150, -140])
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1900.0
              x: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00  1.500e+02
                   7.000e+01]
            nit: 5
          lower:  residual: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00
                              1.500e+02  7.000e+01]
                 marginals: [ 0.000e+00  1.000e+00  0.000e+00  2.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [-3.000e+00 -0.000e+00 -5.00

In [279]:
# Задание 6.2
x = cvxpy.Variable(shape=(5,5), boolean=True)
A = np.array([
    [1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2],
    [10, 3, 1000, 6, 20],
    [19, 7, 6, 1000, 4],
    [8, 2, 20, 4, 1000]
])
constraint_1 = (sum(x) == np.ones(5))
constraint_2 = (sum(x.T) == np.ones(5))
cost = (cvxpy.multiply(A, x)).sum()
problem = cvxpy.Problem(cvxpy.Minimize(cost), constraints=[constraint_1, constraint_2])
print(problem.solve())
print(x.value)
A * x.value

32.0
[[ 0.  0. -0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 [ 1. -0.  0.  0.  0.]
 [ 0.  1.  0.  0. -0.]
 [-0.  0.  0.  1.  0.]]


array([[ 0.,  0., -0.,  0.,  8.],
       [ 0.,  0.,  3.,  0.,  0.],
       [10., -0.,  0.,  0.,  0.],
       [ 0.,  7.,  0.,  0., -0.],
       [-0.,  0.,  0.,  4.,  0.]])

In [310]:
# Задание 6.3
A = np.array([
    [0, 12, 10, 19, 8],
    [12, 0, 3, 7, 2],
    [10, 3, 0, 6, 20],
    [19, 7, 6, 0, 4],
    [8, 2, 20, 4, 0]
])
x = cvxpy.Variable(shape=(5,5), boolean=True)
route = cvxpy.multiply(A, x).sum()
constraints = [(sum(x) == np.ones(5)), (sum(x.T) == np.ones(5))]

for i in range(5):
    for j in range(5):
        constraints.append(x[i][j] + x[j][i] <= 1)

problem = cvxpy.Problem(cvxpy.Minimize(route), constraints=constraints)
print(problem.solve())
print(x.value)
A * x.value

32.0
[[ 0.  0. -0.  0.  1.]
 [-0.  0.  1. -0. -0.]
 [ 1. -0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [-0.  0.  0.  1.  0.]]


array([[ 0.,  0., -0.,  0.,  8.],
       [-0.,  0.,  3., -0., -0.],
       [10., -0.,  0.,  0.,  0.],
       [ 0.,  7.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  4.,  0.]])

In [365]:
# Более общий пример
A = np.array([
        [0, 10, 10, 200, 200, 1], 
        [10, 0, 10, 200, 200, 2], 
        [10, 10, 0, 200, 200, 3], 
        [200, 200, 200, 0, 5, 4], 
        [200, 200, 200, 5, 0, 5],
        [1, 2, 3, 4, 5, 0]
])
x = cvxpy.Variable(shape=A.shape, boolean=True)
route = cvxpy.multiply(A, x).sum()
constraints = []
constraints.append(cvxpy.sum(x, axis=1) == np.ones(A.shape[0]))
constraints.append(cvxpy.sum(x, axis=0) == np.ones(A.shape[0]))

for i in range(5):
    constraints.append(x[i][i] == 0)
    
for i in range(A.shape[0]):
    for j in range(A.shape[0]):
        if i != j:
            constraints.append(x[i, j] + x[j][i] <= 1)

# Нужно добавить условие на связность
        
problem = cvxpy.Problem(cvxpy.Minimize(route), constraints=constraints)
print(problem.solve())
print(x.value)
A * x.value

44.0
[[ 0.  0.  1.  0.  0.  0.]
 [ 1.  0. -0.  0.  0. -0.]
 [-0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -0.]
 [ 0.  0.  0. -0.  0.  1.]
 [-0. -0.  0.  1.  0.  0.]]


array([[ 0.,  0., 10.,  0.,  0.,  0.],
       [10.,  0., -0.,  0.,  0., -0.],
       [-0., 10.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  5., -0.],
       [ 0.,  0.,  0., -0.,  0.,  5.],
       [-0., -0.,  0.,  4.,  0.,  0.]])

# <span style="color:red">Нужно придумать условие на связность!</span>